Cell 1: Setup & Initialization
This cell would contain all your imports (GoogleGenAI, etc.).
It would define a function, let's call it initialize_guild_chat(). This function would:
Read the entire Copilot_Context_Protocol.md file into a string.
Create the ai instance: new GoogleGenAI(...).
Create the "primed" chat instance: ai.chats.create({ systemInstruction: ... }).
Return the chat object.

Cell 2: The Main Chat Loop
This cell would be the interactive part of your application. It would contain a while True: loop.
Inside the loop, it would:
Use Python's input() function to display a prompt like [Operative Kin-Caid]:.
Wait for you to type your message and press Enter.
Take your input and call chat.sendMessage({ message: your_input }).
Get the response from the AI.
Print the response to the screen, formatted nicely, e.g., [Guild AI]: response.text.

In [ ]:
# Cell 1: Setup & Initialization

import os
from google.colab import userdata # Use this for Google Colab
# If not in Colab, manage your API key via another method like python-dotenv
# from dotenv import load_dotenv
# load_dotenv()

# --- Google AI Gemini API Setup ---
# Note: The original prompt used a placeholder 'GoogleGenAI' which is often
# an alias for the main library. The actual library is 'google.generativeai'.
import google.generativeai as genai

# --- Configuration ---
# Ensure your API key is stored as a secret named 'GEMINI_API_KEY' in Colab
# or is available as an environment variable.
try:
    # For Google Colab
    api_key = userdata.get('GEMINI_API_KEY')
except ImportError:
    # For local environments (e.g., using python-dotenv)
    api_key = os.getenv('GEMINI_API_KEY')

if not api_key:
    print("ERROR: GEMINI_API_KEY not found. Please set it as a secret or environment variable.")
else:
    genai.configure(api_key=api_key)

# --- Guild Protocol Functions ---

def load_system_instruction(protocol_file_path="_Admin & Core Docs\Protocols-Frameworks\Copilot_Context_Protocol.md"): 
    """Loads the Guild's core protocol document to be used as the system instruction."""
    try:
        with open(protocol_file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        print(f"ERROR: Protocol file not found at '{protocol_file_path}'.")
        print("Please ensure 'Copilot_Context_Protocol.md' is in the root directory.")
        return None

def initialize_guild_chat(system_instruction):
    """Initializes a new chat session with the Guild's system instruction."""
    if not system_instruction:
        return None
    
    model = genai.GenerativeModel(
        model_name="gemini-2.5-flash-preview-05-20", # Using a recommended modern model
        system_instruction=system_instruction
    )
    chat = model.start_chat()
    return chat

# --- Main Initialization Step ---
print("🛡️  Initializing Guild Interface AI...")
guild_system_protocol = load_system_instruction()

if guild_system_protocol:
    guild_chat_session = initialize_guild_chat(guild_system_protocol)
    if guild_chat_session:
        print("✅  Initialization Complete. AI is calibrated to Guild Protocols.")
        print("➡️  Proceed to run Cell 2 to begin the chat session.")
else:
    print("🔴  Initialization Failed. Please check errors above.")

In [ ]:
# Cell 2: The Main Chat Loop

# This check ensures that the notebook isn't run out of order.
if 'guild_chat_session' not in locals() or not guild_chat_session:
    print("🔴 ERROR: Chat session not initialized. Please run Cell 1 first.")
else:
    print("✅  Chat Session Active. Type 'exit' or 'quit' to end.")
    print("-" * 50)

    while True:
        try:
            # Get input from the Operative
            prompt = input("🔵 [Operative Kin-Caid]: ")

            if prompt.lower() in ['exit', 'quit']:
                print("⚫️  [Guild AI]: Session terminated. Standby for new directive.")
                break

            # Send the message to the Gemini model
            print("⚫️  [Guild AI]: Compiling response...")
            response = guild_chat_session.send_message(prompt)
            
            # Print the AI's response
            print(f"🟢 [Guild AI]: {response.text}")
            print("-" * 20)

        except KeyboardInterrupt:
            print("\n⚫️  [Guild AI]: Kernel interrupted. Session terminated.")
            break
        except Exception as e:
            print(f"🔴 [ERROR]: An unexpected error occurred: {e}")
            break

2. The User Experience (How You Would Use It):
You open the guild_interface_v1.ipynb notebook in JupyterLab or VS Code.
You run the first cell (Setup & Initialization) once. This "boots up" your specialized AI assistant and loads it into memory.
You run the second cell (The Main Chat Loop). It will display the [Operative Kin-Caid]: prompt and wait.
You type your questions directly into the output of that cell.
The conversation happens right there in the notebook's output. The while loop keeps it running, so you can have a long, continuous conversation.
When you're done, you simply interrupt the kernel to stop the loop.